In [5]:
import anthropic
from dotenv import load_dotenv

load_dotenv("../../examples/.env")

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    # api_key="my_api_key",
)
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[{"role": "user", "content": "Hello, Claude"}],
)
print(message.content)

[TextBlock(text="Hello! It's nice to meet you. How can I assist you today?", type='text')]


In [8]:
import anthropic
from pprint import pprint

client = anthropic.Anthropic()
from composio import App
from composio_claude import ComposioToolset

toolset = ComposioToolset()
composio_tools = toolset.get_tools(tools=App.GITHUB)
# pprint(out)


response = client.beta.tools.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    tools=composio_tools,
    messages=[{"role": "user", "content": "Star me composiohq/composio repo in github."}],
)
print(response)

/Users/sawradip/miniconda3/envs/composio-trial/lib/python3.11/site-packages/composio/sdk/sdk.py:344: UserWarning: Using all the actions of an app is not recommended. Please use tags to filter actions or provide specific actions. We just pass the important actions to the agent, but this is not meant to be used in production.
  warnings.warn(


ToolsBetaMessage(id='msg_013yCJVvGvm2j4w7NQoiQLfp', content=[TextBlock(text='<thinking>\nThe relevant tool to star a repository is github_star_repo. This function takes two required parameters:\n\nowner: The owner of the repository. The user provided this as "sawradip".\nrepo: The name of the repository. The user provided this as "sawradip".\n\nSince the user directly provided values for both required parameters, we can proceed with calling the github_star_repo function.\n</thinking>', type='text'), ToolUseBlock(id='toolu_01PKPV6wrQ3p3KpJxGSyfraM', input={'owner': 'sawradip', 'repo': 'sawradip'}, name='github_star_repo', type='tool_use')], model='claude-3-opus-20240229', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=1561, output_tokens=166))


In [9]:
toolset.handle_tool_calls(llm_response=response)

[{'execution_details': {'executed': True}, 'response_data': ''}]

In [6]:
print(response.content.__len__())

2


In [8]:
print(response.content[0].text)

<thinking>
The get_weather tool seems most relevant to answer this request about the weather in a specific location. It requires one parameter:

location: The user provided this - "San Francisco". 

Since the required parameter is provided, we can call the tool to get the current weather for San Francisco.
</thinking>


In [10]:
print(response.content[1].name)

get_weather


In [42]:
from anthropic.types.beta.tools import ToolUseBlock

for content in response.content:
    if isinstance(content, ToolUseBlock):
        action_name_to_execute = content.name
        arguments = content.input
es

{'location': 'San Francisco, CA'}


In [36]:
def handle_tool_calls(
    self, llm_response: ChatCompletion, entity_id: str = "default"
) -> list[any]:
    output = []
    entity = self.client.sdk.get_entity(entity_id)
    try:
        if llm_response.choices:
            for choice in llm_response.choices:
                if choice.message.tool_calls:
                    for tool_call in choice.message.tool_calls:
                        action_name_to_execute = tool_call.function.name
                        action = self.client.sdk.get_action_enum_without_tool(
                            action_name=action_name_to_execute
                        )
                        arguments = json.loads(tool_call.function.arguments)
                        account = entity.get_connection(app_name=action.service)
                        output.append(account.execute_action(action, arguments))

    except Exception as e:
        raise e from e

    return output

NameError: name 'ChatCompletion' is not defined

In [11]:
print(response.content[1].input)

{'location': 'San Francisco, CA'}


In [24]:
import json
import dotenv
from pprint import pprint
from openai import OpenAI
from composio_openai import App, ComposioToolset


# Loading the variables from .env file
dotenv.load_dotenv()

client = OpenAI()


# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {
            "role": "user",
            "content": "What's the weather like in San Francisco, Tokyo, and Paris?",
        }
    ],
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)

pprint(response)

# result = toolset.handle_tool_calls(response)
# pprint(result)

ChatCompletion(id='chatcmpl-9Km8DNrJxFM42yy1Xfp4IkzYLqTNU', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_nR01GHOUILL6uprt1KMuYleu', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_TQVtvYkLp6HGBU8Gk9XdbuC4', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_ik82gT8YyRbKE2ckVcG1rTB4', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]))], created=1714738605, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3b956da36b', usage=CompletionUsage(completion_tokens=77, prompt_tokens=88, total_tokens=165))


In [27]:
print(response.choices[0].message.function_call)

None


In [31]:
print(response.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_nR01GHOUILL6uprt1KMuYleu', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_TQVtvYkLp6HGBU8Gk9XdbuC4', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_ik82gT8YyRbKE2ckVcG1rTB4', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]
